In [1]:
# This cell sets the CWD to the parent directory.  
# If you run this more than once, it will cause problems!

import os
wd = os.getcwd()
os.chdir('/'.join(wd.split('/')[:-1])+'/'  )
print("CWD:" + os.getcwd())




CWD:/Users/gjdpci/Dropbox/Code/Catching - IPD/expansion analysis - UXF 1/Interception_UXF_Analysis


In [2]:
import sys

sys.path.append("Modules/")
# sys.path.append("../Modules/")
sys.path.append("/")


import logging
import pickle
import numpy as np
import pandas as pd


fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from loadData import unpackSession

sys.path.append("pyFiles/")
from processData import *

CWD:/Users/gjdpci/Dropbox/Code/Catching - IPD/expansion analysis - UXF 1/Interception_UXF_Analysis


### Import raw subject data (slow), or previously imported data from file (fast).

If this is the first time you're running this code, it will take a little bit to processe each trial.
However, when it is done, the results are saved to a pickle file.  

The "doNotLoad" is False by default.
When False, the method will always check if this pickle file exists.  If it does, it loads it.  This is a much faster process.

If you want to load the raw data (start from scratch), set doNotLoad=True


In [3]:
subNum = 4

sessionDict = unpackSession(subNum,doNotLoad=False)

INFO_loadData-unpackSession(): - Processing session: Data/P_200917094202
INFO_loadData-unpackSession(): - Importing session dict from pickle.


0: P_201218121321_sub1
1: P_201218121321_sub1c
2: P_201218121321_sub1.zip
3: P_201218121321_sub1b
***> 4: P_200917094202
5: P_201219105516


### Let's inspect the sessiondict and see how the data is organized.

In [4]:
list(sessionDict.keys())

['subID',
 'trialInfo',
 'expConfig',
 'rawExpUnity',
 'rawExpGaze',
 'processedExp',
 'rawCalibUnity',
 'rawCalibGaze',
 'processedCalib',
 'analysisParameters']

### A description of what's in the session file:

* subID: self explanatory
* trialInfo: metadata for the trial
* expConfig: metadata for the experiment

* rawExpUnity: raw data recorded at each Unity call ot Update() - 90 Hz on the Vive.  Data is for catching experiment trials only.

* rawExpGaze: raw data recorded at each sample of a Pupil eye camera - [two interleaved 120 hz streams, so approx 240 hz] Data is for catching experiment trials only.

* processedExp: Formed by upsampling rawExpUnity to match the frequency of rawExpGaze, and merging. Data is for catching experiment trials only.

* rawCalibUnity: Same as rawExpUnity but for calibraiton assessment trials only.
* rawCalibGaze: Same as rawExpGaze but for calibraiton assessment trials only.
* processedCalib: Same as processedExp but for calibraiton assessment trials only.

### Let's poke around the trialInfo metadata

In [5]:
sessionDict['trialInfo'].keys()

MultiIndex([(                 'ballFinalPos', 'x'),
            (                 'ballFinalPos', 'y'),
            (                 'ballFinalPos', 'z'),
            (               'ballInitialPos', 'x'),
            (               'ballInitialPos', 'y'),
            (               'ballInitialPos', 'z'),
            (               'ballInitialVel', 'x'),
            (               'ballInitialVel', 'y'),
            (               'ballInitialVel', 'z'),
            (       'ball_movement_filename',  ''),
            (                  'blockNumber',  ''),
            (     'camera_movement_filename',  ''),
            (           'contactLocOnPaddle', 'x'),
            (           'contactLocOnPaddle', 'y'),
            (           'contactLocOnPaddle', 'z'),
            (            'contactLocinWorld', 'x'),
            (            'contactLocinWorld', 'y'),
            (            'contactLocinWorld', 'z'),
            (                    'directory',  ''),
            

Let's get some values from the metadata for a single trial:

In [6]:
trialRowIdx = 10

aTrialsInfo = sessionDict['trialInfo'].loc[trialRowIdx]

print('Trial number: {tNum} \nTrial type: {tType}'.format(tNum = int(aTrialsInfo['trialNumber']),
                                                   tType = str(aTrialsInfo['trialType'].values)))


Trial number: 11 
Trial type: ['interception']


Now, let's have a look at what kind of per-frame data in associated with this trial.  You have access to the raw data in rawUnity and rawGaze ['processedExp'] associated with this trial.

In [7]:
sessionDict['rawCalibGaze'].keys()

Index(['base_data', 'confidence', 'eye_center0_3d_x', 'eye_center0_3d_y',
       'eye_center0_3d_z', 'eye_center1_3d_x', 'eye_center1_3d_y',
       'eye_center1_3d_z', 'gaze_normal0_x', 'gaze_normal0_y',
       'gaze_normal0_z', 'gaze_normal1_x', 'gaze_normal1_y', 'gaze_normal1_z',
       'gaze_point_3d_x', 'gaze_point_3d_y', 'gaze_point_3d_z', 'norm_pos_x',
       'norm_pos_y', 'pupilTimestamp', 'world_index'],
      dtype='object')

In [8]:
sessionDict['processedCalib'].keys()
sessionDict['processedCalib']['camera']

MultiIndex([('ballColliderRadius',     ''),
            (    'ballMeshRadius',     ''),
            (           'ballPos',    'x'),
            (           'ballPos',    'y'),
            (           'ballPos',    'z'),
            (           'ballRot',    'x'),
            (           'ballRot',    'y'),
            (           'ballRot',    'z'),
            (           'ballVel',    'x'),
            (           'ballVel',    'y'),
            (           'ballVel',    'z'),
            (         'base_data',     ''),
            (       'blockNumber',     ''),
            (            'camera', 'R0C0'),
            (            'camera', 'R0C1'),
            (            'camera', 'R0C2'),
            (            'camera', 'R0C3'),
            (            'camera', 'R1C0'),
            (            'camera', 'R1C1'),
            (            'camera', 'R1C2'),
            (            'camera', 'R1C3'),
            (            'camera', 'R2C0'),
            (            'camera

Note that the column indices (listed above) are 'multiIndex'.  
They complicate things and can cause issues sometimes, but are generally helpful for data organization. 

Using the first-level column index will also pull up all subindices:

In [9]:
sessionDict['processedCalib']

ballColliderRadius ballMeshRadius ballPos         ballRot          \
                                             x   y   z       x   y   z   
0                   NaN            NaN     NaN NaN NaN     NaN NaN NaN   
1                   NaN            NaN     NaN NaN NaN     NaN NaN NaN   
2                   NaN            NaN     NaN NaN NaN     NaN NaN NaN   
3                   NaN            NaN     NaN NaN NaN     NaN NaN NaN   
4                   NaN            NaN     NaN NaN NaN     NaN NaN NaN   
...                 ...            ...     ...  ..  ..     ...  ..  ..   
2934                NaN            NaN     NaN NaN NaN     NaN NaN NaN   
2935                NaN            NaN     NaN NaN NaN     NaN NaN NaN   
2936                NaN            NaN     NaN NaN NaN     NaN NaN NaN   
2937                NaN            NaN     NaN NaN NaN     NaN NaN NaN   
2938                NaN            NaN     NaN NaN NaN     NaN NaN NaN   

     ballVel      ... paddle paddlePos       paddleRot        pupilTimestamp  \
           x   y  ...   R3C3         x  y  z         x  y   z                  
0        NaN NaN  ...      1         0  0  0         0  0  90  483821.611892   
1        NaN NaN  ...      1         0  0  0         0  0  90  483821.614828   
2        NaN NaN  ...      1         0  0  0         0  0  90  483821.618226   
3        NaN NaN  ...      1         0  0  0         0  0  90  483821.621605   
4        NaN NaN  ...      1         0  0  0         0  0  90  483821.625638   
...      ...  ..  ...    ...       ... .. ..       ... ..  ..            ...   
2934     NaN NaN  ...      1         0  0  0         0  0  90  483838.934642   
2935     NaN NaN  ...      1         0  0  0         0  0  90  483838.937650   
2936     NaN NaN  ...      1         0  0  0         0  0  90  483838.941609   
2937     NaN NaN  ...      1         0  0  0         0  0  90  483838.945445   
2938     NaN NaN  ...      1         0  0  0         0  0  90  483838.945897   

     trialNumber world_index  
                              
0              1         NaN  
1              1      1393.0  
2              1      1393.0  
3              1      1393.0  
4              1      1394.0  
...          ...         ...  
2934           9      1912.0  
2935           9      1912.0  
2936           9      1912.0  
2937           9      1912.0  
2938           9      1912.0  

[2939 rows x 80 columns]

In [10]:
sessionDict['processedCalib']['gaze-normal0'].head(10) # head(10) shows only teh first ten values

,x,y,z
0,NaN,NaN,NaN
1,-0.168069,-0.205025,0.964219
2,-0.168069,-0.205025,0.964219
3,-0.167594,-0.203713,0.964579
4,-0.167594,-0.203713,0.964579
5,-0.168906,-0.203879,0.964314
6,-0.170218,-0.204046,0.964049
7,-0.168410,-0.205108,0.964142
8,-0.168410,-0.205108,0.964142
9,-0.168410,-0.205108,0.964142


Use a tuple to take advantage of multiindices:

In [11]:
sessionDict['processedCalib'][('gaze-normal0','x')].head(10) # head(10) shows only teh first ten values

0         NaN
1   -0.168069
2   -0.168069
3   -0.167594
4   -0.167594
5   -0.168906
6   -0.170218
7   -0.168410
8   -0.168410
9   -0.168410
Name: (gaze-normal0, x), dtype: float64

In [12]:
sessionDict['processedCalib'][('gaze-normal0','x')].head(10)

0         NaN
1   -0.168069
2   -0.168069
3   -0.167594
4   -0.167594
5   -0.168906
6   -0.170218
7   -0.168410
8   -0.168410
9   -0.168410
Name: (gaze-normal0, x), dtype: float64

# Some approaches to computation...

Summary statistics are easy!

In [13]:
sessionDict['processedCalib'][('gaze-normal0','x')].mean()

0.0570288521705389

### Want to compute a new measurement or metric per frame?   

You probably want to iterate through each frame/row of processedExp or processedCalib and use the existing data to calculate a new measure. 

Below, I apply "anonymous" function to each row of [('gaze-normal0','x')] to multiply it by two.

In [14]:
sessionDict['processedCalib'][('gaze-normal0','x')].apply(lambda row: row*2)

0            NaN
1      -0.336139
2      -0.336139
3      -0.335188
4      -0.335188
          ...   
2934    0.572848
2935    0.572848
2936    0.572848
2937    0.572879
2938    0.572911
Name: (gaze-normal0, x), Length: 2939, dtype: float64

### We can get a bit more tricky here, too.  
For example, we can apply a custom function to normalize the gaze-normal0 vector.
Note the axis argin, which makes sure that we're applying this function to each ROW (and not column, where axis=0)

You may get the error: "RuntimeWarning: invalid value encountered in true_divide"
This is caused by a divide by zero.

In [15]:
def normalizeVector(xyz):
    '''
    Input be a 3 element array of containing the x,y,and z data of a 3D vector.
    Returns a normalized 3 element array
    '''
    
    # Sometimes necessary.
    xyz = np.array(xyz)
    xyz = xyz / np.linalg.norm(xyz)
    return xyz 

sessionDict['processedCalib']['gaze-normal0'].apply(lambda row: normalizeVector(row),axis=1)


0                                         [nan, nan, nan]
1       [-0.16806947930037447, -0.2050245670152353, 0....
2       [-0.16806947930037447, -0.2050245670152353, 0....
3       [-0.16759414157050426, -0.20371301766216995, 0...
4       [-0.16759414157050426, -0.20371301766216995, 0...
                              ...                        
2934    [0.28642385184979485, 0.11325367216106358, 0.9...
2935    [0.28642385184979485, 0.11325367216106358, 0.9...
2936    [0.28642385184979485, 0.11325367216106358, 0.9...
2937    [0.2864396384906077, 0.11359014237079905, 0.95...
2938    [0.286455392054954, 0.1139265994637726, 0.9512...
Length: 2939, dtype: object

### It's a very good idea to assume your "normals" are not actually normalized.  After all, the time-series data was merged, upsampled, and interpolated!

# What if we cant to calculate something per trial?

You can also compute by applying a method to each "trial" processedCalib, mwhere each trial is a single fixation, or a sequence of fixations and saccades, or a VOR ( every time the black dot turned yellow, that was a single trial). 

In the case of the ball catching task, a single trial was one throw of the ball. That data is stored in processedExp, and we won't worry about it now..

### The processed dataframes can be sliced into trials using the groupby function.

Lets first create the groupby object.  

In [16]:
gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])

Group keys refer to the trial number

In [17]:
gbProcessedCalib_trial.groups.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9])

Each group (in this case, each trial) is actually a dataframe "slice" of the rows of processedCalib that match that trial number.

In [18]:
gbProcessedCalib_trial.get_group(10)

KeyError: 10


### Trials / groups are useful because they are iterable.

You can iterate through rows of trialInfoDF in a number of ways, but I will demonstrate one way that makes it easy to get both trial metadata and per-frame data from the processed dataframe

Here's the method done manually.

In [ ]:
trialRowIdx = 10

trMetaData =  sessionDict['trialInfo'].iloc[10]

# Note that trial numbers start at 1, not 0. 
# Trial number and trialRowIdx index are not the same thing!
print('Trial number: {tNum}'.format(tNum = int(trMetaData['trialNumber'])))

# This dataframe contains the per-frame processed data associated with this trial
procDF = gbProcessedCalib_trial.get_group(int(trMetaData['trialNumber']))

# Is the target on trial head fixed the entire time?
# len(procDF['isHeadFixed'].drop_duplicates()) # If all values are true, then yes!
if ( sum(procDF['isHeadFixed'] == True) == len(procDF) ):
    print('This trial is a fixation or fixation+saccade trial.')


Now, let's take the same general approach using iteration

In [ ]:
for trialRowIdx, trMetaData in sessionDict['trialInfo'].iterrows():
    
    # This dataframe contains the per-frame processed data associated with this trial
    procDF = gbProcessedCalib_trial.get_group(int(trMetaData['trialNumber']))
    
    targetType = []

    
    if ( sum(procDF['isHeadFixed'] == False) ==  len(procDF) ):
        
        targetType = 'VOR'
    
    elif( sum(procDF['isHeadFixed'] == True) == len(procDF) ):
        
        # Count the number of target positions within the local space (head-centered space)
        if( len(procDF['targeLocalPos'].drop_duplicates()) == 1 ):
            # Only one target, so it's a fixation trial
            targetType = 'fixation'
        else:
            # multiple targets, so it's a saccade trial
            targetType = 'fixation+saccade'
            
    else:
        # The trial has both head fixed and world fixed targets.  
        # We did not plan for that, so let's label it as "unknown."
        
        targetType = 'unknown'
        
    print('Trial number: {tNum}, type: {tType}'.format(tNum = int(trMetaData['trialNumber']),
                                                   tType = targetType))


### Nice!  Now, how about a more modular approach?

In [ ]:
def findCalibrationTargetType(sessionIn):
    '''
    Input: Session dictionary
    Output:  Session dictionary with new column sessionDict['trialInfo']['targetType']
    '''
    
    gbProcessedCalib_trial = sessionIn['processedCalib'].groupby(['trialNumber'])
    
    targetTypes = []
    for trialRowIdx, trMetaData in sessionIn['trialInfo'].iterrows():

        # This dataframe contains the per-frame processed data associated with this trial
        procDF = gbProcessedCalib_trial.get_group(int(trMetaData['trialNumber']))

        targetType = []


        if ( sum(procDF['isHeadFixed'] == False) ==  len(procDF) ):

            targetType = 'VOR'

        elif( sum(procDF['isHeadFixed'] == True) == len(procDF) ):

            # Count the number of target positions within the local space (head-centered space)
            if( len(procDF['targeLocalPos'].drop_duplicates()) == 1 ):
                # Only one target, so it's a fixation trial
                targetType = 'fixation'
            else:
                # multiple targets, so it's a saccade trial
                targetType = 'fixation+saccade'

        else:
            # The trial has both head fixed and world fixed targets.  
            # We did not plan for that, so let's label it as "unknown."

            targetType = 'unknown'

        print('Trial number: {tNum}, type: {tType}'.format(tNum = int(trMetaData['trialNumber']),
                                                       tType = targetType))
        targetTypes.append(targetType)
        
    
    sessionIn['trialInfo']['targetType'] = targetTypes
    
    logger.info('Added sessionDict[\'trialInfo\'][\'targetType\']')
    
    return sessionDict


In [ ]:
sessionDict = findCalibrationTargetType(sessionDict)

Wonderful.  
The idea here is to build up a bunch of methods that can be called in a linear fashion.  
Most will perform number crunching and add new columns, or modify old ones (athough, never change the raw data!)
Eventually, some will be plotting functions.

I've taken this approach to processing ball catching data.  To see this approach in practice, open up processDataE1,py, and see processData().  

# Upwards and onwards!

In [ ]:
sessionDict['processedCalib'].keys()

In [ ]:
# sessionDict['trialInfo'].keys()

In [ ]:
gbProcessedCalib_trial = sessionDict['processedCalib'].groupby(['trialNumber'])
perFrameTrialData = gbProcessedCalib_trial.get_group(14)
np.unique(perFrameTrialData['targeLocalPos'],axis=0)

In [ ]:
perFrameTrialData['gaze-normal1'].values
perFrameTrialData['gaze-normal1'].plot()

# np.array([np.divide(XYZ,np.linalg.norm(XYZ)) for XYZ in perFrameTrialData['gaze-normal0'].values],dtype=np.float)

In [ ]:
# np.array([np.divide(XYZ,np.linalg.norm(XYZ)) for XYZ in paddleToBallVec_fr_XYZ],dtype=np.float)

In [ ]:
# ['trialInfo']['targetType']

In [ ]:
paddleToBallDir_fr_XYZ = np.array([np.divide(XYZ,np.linalg.norm(XYZ)) for XYZ in paddleToBallVec_fr_XYZ],dtype=np.float)